In [75]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from tensorflow import keras
from tensorflow.keras import layers

In [76]:
#Charge the data
train_data_df = pd.read_csv(r'C:\Repositorio\Titanic competition\Data\train.csv')
test_data_df = pd.read_csv(r'C:\Repositorio\Titanic competition\Data\test.csv')

In [77]:
#Prepare the data
train_data_df.drop(['Name','Ticket','Cabin'], axis=1, inplace=True)
test_data_df.drop(['Name','Ticket','Cabin'], axis=1, inplace=True)

y = train_data_df['Survived']
test_X = test_data_df
train_X, val_X, train_y, val_y = train_test_split(train_data_df, y, random_state = 0)
train_X.drop(['Survived'], axis=1, inplace=True)
val_X.drop(['Survived'], axis=1, inplace=True)


# Obtener los tipos de datos categoricos
s = (train_X.dtypes == 'object')
object_cols = list(s[s].index)





#Apply one-hot encoding to each column with categorical data and reset index
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(train_X[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(val_X[object_cols]))
OH_cols_test = pd.DataFrame(OH_encoder.transform(test_X[object_cols]))
OH_cols_train.index = train_X.index
OH_cols_valid.index = val_X.index
OH_cols_test.index = test_X.index
# Remove categorical columns (will replace with one-hot encoding)
num_X_train = train_X.drop(object_cols, axis=1)
num_X_valid = val_X.drop(object_cols, axis=1)
num_X_test = test_X.drop(object_cols, axis=1)
# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)
OH_X_test = pd.concat([num_X_test, OH_cols_test], axis=1)
#Fill the dataframes NaN
OH_X_train = OH_X_train.fillna(method='bfill', axis=0).fillna(0)
OH_X_valid = OH_X_valid.fillna(method='bfill', axis=0).fillna(0)
val_y = val_y.fillna(method='bfill', axis=0).fillna(0)
train_y = train_y.fillna(method='bfill', axis=0).fillna(0)



#NOTA: HACER SEPARACION DE VALORES CATEGORICOS PARA OBETENER COLUMNA POR CADA UNA

print("Categorical variables:")
print(object_cols)

C:\Python39\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
Categorical variables:
['Sex', 'Embarked']


In [78]:
#Select & fit a model
titanic_model = XGBRegressor(n_estimators=500, learning_rate=0.05)
titanic_model.fit(OH_X_train, train_y,
                  early_stopping_rounds=1,
                  eval_set=[(OH_X_valid, val_y)])

[0]	validation_0-rmse:0.48747
[1]	validation_0-rmse:0.47581
[2]	validation_0-rmse:0.46484
[3]	validation_0-rmse:0.45473
[4]	validation_0-rmse:0.44593
[5]	validation_0-rmse:0.43789
[6]	validation_0-rmse:0.43095
[7]	validation_0-rmse:0.42411
[8]	validation_0-rmse:0.41827
[9]	validation_0-rmse:0.41269
[10]	validation_0-rmse:0.40764
[11]	validation_0-rmse:0.40322
[12]	validation_0-rmse:0.39922
[13]	validation_0-rmse:0.39580
[14]	validation_0-rmse:0.39240
[15]	validation_0-rmse:0.38943
[16]	validation_0-rmse:0.38702
[17]	validation_0-rmse:0.38416
[18]	validation_0-rmse:0.38162
[19]	validation_0-rmse:0.37937
[20]	validation_0-rmse:0.37709
[21]	validation_0-rmse:0.37557
[22]	validation_0-rmse:0.37450
[23]	validation_0-rmse:0.37346
[24]	validation_0-rmse:0.37185
[25]	validation_0-rmse:0.37049
[26]	validation_0-rmse:0.36960
[27]	validation_0-rmse:0.36764
[28]	validation_0-rmse:0.36710
[29]	validation_0-rmse:0.36600
[30]	validation_0-rmse:0.36577
[31]	validation_0-rmse:0.36502
[32]	validation_0-

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=500, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [79]:
#Evaluate the model(XG_Boost model)

predictions = titanic_model.predict(OH_X_valid)
print("Mean Absolute Error: " + str(mean_absolute_error(val_y,predictions)))

Mean Absolute Error: 0.2801858525171943


In [117]:
#Create submission file
final_prediction = titanic_model.predict(OH_X_test)#problema de orden??? reordenar columnas
for numer_p,passenger in enumerate(final_prediction):
    if passenger>0.5:
        final_prediction[numer_p]=int(1)
    else:
        final_prediction[numer_p]=int(0)
final_prediction = final_prediction.astype(int)
gender_submission_df = pd.DataFrame()
gender_submission_df['PassengerId'] = OH_X_test['PassengerId']
gender_submission_df['Survived'] = final_prediction


print(final_prediction)
gender_submission_df.to_csv(r'C:\Repositorio\Titanic competition\Results\gender_submission.csv', index=False)

[0 0 0 0 1 0 0 0 1 0 0 0 1 0 1 1 0 0 0 1 0 1 0 0 1 0 1 0 1 0 0 0 1 1 0 0 0
 0 0 0 0 0 0 1 1 0 1 0 1 1 0 0 1 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 1 0 0 0 1
 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 1 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 1 1 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0
 0 0 1 0 0 1 0 0 1 0 1 1 0 1 1 0 0 1 0 0 0 1 0 0 0 0 0 1 1 0 1 1 0 0 1 0 1
 0 1 0 1 0 0 0 1 0 1 0 1 0 0 0 1 1 0 1 0 0 1 0 1 0 0 0 0 1 0 0 0 0 1 0 1 0
 1 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 1 0 1 0 1 1 0 0 0 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 1 1 0
 1 0 0 0 0 0 0 0 0 1 1 1 0 0 1 0 1 1 0 0 0 1 0 1 0 0 1 0 1 1 0 1 1 0 0 1 0
 0 1 0 0 1 1 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 1 0 0 1 0 1 0 0 1 0 1 0 1 0 0
 0 0 1 0 1 0 0 1 0 0 1]


In [118]:
#Evaluate the model with Keras model

#Define the  model
model = keras.Sequential([
    layers.BatchNormalization(),
    layers.Dense(256, activation='relu', input_shape=[12]),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid'),
])
#Add the cross-entropy loss & accuracy metric
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['binary_accuracy'],
)
early_stopping = keras.callbacks.EarlyStopping(
    patience=10,
    min_delta=0.001,
    restore_best_weights=True,
)
#Fit the model
history = model.fit(
    OH_X_train, train_y,
    validation_data=(OH_X_valid, val_y),
    batch_size=512,
    epochs=1000,
    callbacks=[early_stopping],
   # hide the output because we have so many epochs
)


Epoch 1/1000
2/2 [==============================] - 8s 715ms/step - loss: 0.7843 - binary_accuracy: 0.5988 - val_loss: 0.6423 - val_binary_accuracy: 0.6816
Epoch 2/1000
2/2 [==============================] - 0s 71ms/step - loss: 0.5389 - binary_accuracy: 0.7725 - val_loss: 0.6246 - val_binary_accuracy: 0.6771
Epoch 3/1000
2/2 [==============================] - 0s 67ms/step - loss: 0.5558 - binary_accuracy: 0.7904 - val_loss: 0.6220 - val_binary_accuracy: 0.6682
Epoch 4/1000
2/2 [==============================] - 0s 64ms/step - loss: 0.5439 - binary_accuracy: 0.7889 - val_loss: 0.6103 - val_binary_accuracy: 0.6771
Epoch 5/1000
2/2 [==============================] - 0s 66ms/step - loss: 0.5033 - binary_accuracy: 0.8054 - val_loss: 0.5950 - val_binary_accuracy: 0.6816
Epoch 6/1000
2/2 [==============================] - 0s 76ms/step - loss: 0.4839 - binary_accuracy: 0.8114 - val_loss: 0.5850 - val_binary_accuracy: 0.6951
Epoch 7/1000
2/2 [==============================] - 0s 60ms/step - lo

In [119]:
deep_predictions = model.predict(OH_X_valid)
print(deep_predictions)

[[0.3414706 ]
 [0.34164828]
 [0.22304761]
 [0.5846433 ]
 [0.26053694]
 [0.32579562]
 [0.69423723]
 [0.5438819 ]
 [0.45015213]
 [0.3073522 ]
 [0.2706874 ]
 [0.36868635]
 [0.3279963 ]
 [0.40240481]
 [0.5874909 ]
 [0.37225825]
 [0.28606522]
 [0.25334895]
 [0.25333554]
 [0.36720765]
 [0.32868463]
 [0.7127599 ]
 [0.26817966]
 [0.3618184 ]
 [0.3360873 ]
 [0.5497807 ]
 [0.2601512 ]
 [0.34533703]
 [0.41135943]
 [0.31684548]
 [0.23892942]
 [0.34258038]
 [0.26864845]
 [0.34769624]
 [0.25038064]
 [0.35890257]
 [0.23359326]
 [0.3369795 ]
 [0.33367148]
 [0.25873134]
 [0.28145948]
 [0.27421498]
 [0.32586205]
 [0.25883776]
 [0.52555335]
 [0.3199016 ]
 [0.30420226]
 [0.57757986]
 [0.28426462]
 [0.29924107]
 [0.42900947]
 [0.3390622 ]
 [0.45170534]
 [0.2991704 ]
 [0.42379016]
 [0.38197836]
 [0.30439758]
 [0.24706665]
 [0.27180618]
 [0.23552564]
 [0.31908035]
 [0.22818142]
 [0.49183497]
 [0.42576793]
 [0.27719098]
 [0.24822578]
 [0.41188082]
 [0.34313127]
 [0.42728063]
 [0.52732605]
 [0.3619238 ]
 [0.30

In [ ]:
#Create submission file
deep_predictions = titanic_model.predict(OH_X_test)#problema de orden??? reordenar columnas
for numer_p,passenger in enumerate(deep_predictions):
    if passenger>0.5:
        deep_predictions[numer_p]=int(1)
    else:
        deep_predictions[numer_p]=int(0)
deep_predictions = final_prediction.astype(int)
gender_submission_df = pd.DataFrame()
gender_submission_df['PassengerId'] = OH_X_test['PassengerId']
gender_submission_df['Survived'] = deep_predictions


gender_submission_df.to_csv(r'C:\Repositorio\Titanic competition\Results\Deep_Results\gender_submission.csv', index=False)